## Расчет давления и температуры из кодов.

Для сенсора давления HP03SB

In [29]:
!pwd

/home/alena/SPHERE/Analysis/Collect_datum


In [83]:
# calculate values of pressure & temperature from kods pp & tt
# pp - pressure kod, tt - temperature kod  in INT numbers

def calc_bar_temp(pp, tt, ptype):

    ## set formula parameters
    a  = [4, 7., 16., 6, 13]
    c  = [7.0, 13919.0, 1793.0, 198.0, 131.0, 35820.0, 5066.0, 2500.0]  ## C0 onboard 0
    c1 = [7.0, 16762.0, 2083.0, 310.0, 750.0, 35424.0, 5338.0, 2500.0]  ## C1 onboard 1
    c2 = [7.0, 27626.0, 3949.0, 314.0, 317.0, 30355.0, 5441.0, 2500.0]  ## C2 ground 0
    c3 = [7.0, 18489.0, 2736.0, 292.0, 356.0, 35569.0, 5253.0, 2500.0]  ## C3 ground 1
    
    if   ptype == 0:   ## barometer onboard 1
        c = c
    elif ptype == 1:   ## barometer onboard 1
        c = c1
    elif ptype == 2: ## barometer ground 0 
        c = c2
    elif ptype == 3: ## barometer ground 0
        c = c3
    elif ptype == 10: ## example barometer
        c = [7, 29908, 3724, 312, 441, 9191, 3990, 2500]
        a = [4, 1, 4, 4, 9]
    else:
        print("\n\n Error! Unknown coefficient set!!! \n")
        return
     
    ## -- 1 step: --  
    dUT  = tt - c[5]
    qqq  = dUT * dUT / (128.0 * 128.0 * (2 ** a[3])) 
    if(tt < c[5]):
        qqq *= a[2]
    else:
        qqq *= a[1]
    dUT -= qqq
 
    ### -- step 2 --
    OFF = (c[2] + (c[4] - 1024.0) * dUT / 16384.0 ) * 4
    SENS = c[1] + (c[3] * dUT / 1024.0)
    X = SENS * (pp - 7168) / 16384 - OFF
    P = X * 100.0/ 32.0 + c[7] * 10
    P = P / 10
     
    ### step 3: calculate temperature
    T = 250 + dUT * c[6] / 65536 - dUT/ (2 ** a[4])
    T = T / 10

    print("Bar:  T[ %5i ] = %5.1f C  P[ %5i ] = %6.2f kPa (%4.1f mm w)" % (tt, T, pp, P/100, P/0.981) )
    return P/100, T

# Проверка

In [82]:
# example from datasheet sa and sb
P, T = calc_bar_temp( 30036, 4107, 10) ## right answer is: T= -7.2 P= 991.8 hpa
print("%.2f %.1f" % (P, T))

Bar:  T[  4107 ] =  -7.3 C  P[ 30036 ] =  99.18 kPa (10110.4 mm w)
99.18 -7.3


In [68]:
# on_ground Wed Mar 14 14:31:50 2012
#0 Bar:  T[ 30983 ] = 30.2 C  P[ 29401 ] = 93.64 kPa (9545.1 mm w)
#1 Bar:  T[ 33579 ] = 8.6 C  P[ 38266 ] = 95.87 kPa (9772.5 mm w)
calc_bar_temp( 29401, 30983, 2)
calc_bar_temp( 38266, 33579, 3)

Bar:  T[ 30983 ] =  30.2 C  P[ 29401 ] =  93.94 kPa (9575.8 mm w)
Bar:  T[ 33579 ] =   8.6 C  P[ 38266 ] =  95.95 kPa (9781.2 mm w)


In [69]:
#0 Bar: t[ 30236 ] =  24.0 C P[ 29521 ] = 92.96 KPa (9476.1 mm w)
#1 Bar: t[ 30801 ] = -15.9 C P[ 40285 ] = 95.93 KPa (9778.5 mm w)
calc_bar_temp( 29521, 30236, 2)
calc_bar_temp( 40285, 30801, 3)

Bar:  T[ 30236 ] =  24.0 C  P[ 29521 ] =  93.20 kPa (9500.6 mm w)
Bar:  T[ 30801 ] = -15.9 C  P[ 40285 ] =  95.77 kPa (9762.2 mm w)


# The END

In [ ]:
## функция на си
// calculate values of pressure & temperature from kods pp & tt
// pp - pressure kod, tt - temperature kod         
// in INT numbers
int barometer::calc_bar_temp(unsigned int pp, unsigned int tt, char *message)
{
    double dUT = 0., qqq = 0., ppp = 0.;
    double P  = 0.0, T  = 0.0, OFF = 0.0, SENS = 0.0;
    long double X = 0.0;
    float c[8];
    int a[5];
    unsigned int i = 0;
    //cc[8] = {7, 13919, 1793, 198, 131, 3052, 5066, 2500};
    //float cc[8] = {7, 29908, 3724, 312, 441, 9191, 3990, 2500};
    //int   aa[5] = {4, 1, 4, 4, 9};

    //pp = 42610; tt = 36508;
    //pp = 30036; tt = 4107;
    //printf("tt = %6i pp = %6i\n", tt, pp);
    pp<<=16;
    pp>>=16;
    tt<<=16;
    tt>>=16;
    for(i = 0; i< 8; i++)
    {
        c[i] = (float) cc[i];
        if(i < 5) 
            a[i] = aa[i];
        //printf("%i : cc[i]=%i, c[i]=%f, aa[i]=%i, a[i]=%i\n", i, cc[i], c[i], aa[i], a[i]);
    }

    // -- 1 step: --
    dUT  = (float)tt - c[5];
    //printf("dUT = %f\n", dUT);
    qqq  = dUT * dUT; // 128=2^7
    //printf("qqq = %f\n", qqq);
    if(tt < c[5])
    {
        qqq *= (float) a[2];
        //printf("qqq = %f, a[2]=%i\n", qqq, a[2]);
    }
    else
    {
        qqq *= (float) a[1];
        //printf("qqq = %f, a[1]=%i\n", qqq, a[1]);
    }

    ppp = (128.0 * 128.0 *(float)pow2_x(a[3]) );
    //printf("qqq = %f, ppp = %f\n", qqq, ppp);
    qqq /= ppp;
    dUT -= qqq;
    //printf("dUT = %f\n", dUT);

    qqq = 0;
    T  = dUT * c[6];
    T /= 65536.0; // 65536 = 2^16
    T = T + 250.0;
    qqq = dUT/pow2_x(a[4]);
    T -= qqq;
    //printf(" T = %f\n",  T);

    // -- step 2 --
    OFF = (c[4] - 1024.0) * dUT;
    OFF /= 16384.0;  // 2^14
    OFF = (c[2] + OFF ) * 4.0;
    //printf("OFF = %f\n", OFF);

    SENS = c[1] + (c[3]* dUT / 1024.0);
    //printf("SENS = %lf\n", SENS);

    X = SENS * ((float)pp - 7168.0)/16384.0 - OFF;
    //printf("X = %Lf\n", X);

    P = X * 10.0/ 32.0 + c[7];
    //printf("P = %f\n", P);

    T = T/10;
    P = P/1;//-T;

    sprintf(message, "Bar:  T[ %5i ] = %5.1f C  P[ %5i ] = %6.2f kPa (%4.1f mm w)\n", tt, T, pp, P/100, P/0.981);
    if(dout) fprintf(dout, "\n Bar: P = %.2f kPa  T = %.1f C\n", P/10, T);
}